# Global Greening

## Installing & Importing Libraries

this notebook version is setup so we can train the model on VM workbench

In [1]:
import pickle
import numpy as np
import pandas as pd
import math
import time
from PIL import Image
import json
# from patchify import patchify
# import albumentations as A
from IPython.display import SVG
#import graphviz
import matplotlib.pyplot as plt
%matplotlib inline
import os, re, sys, random, shutil #, cv2
from google.cloud import storage

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras import applications, optimizers
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import model_to_dot, plot_model, image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, LearningRateScheduler
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, ZeroPadding2D, Dropout

from sklearn.preprocessing import MinMaxScaler

/Users/Alenka/.pyenv/versions/3.10.6/envs/GlobalGreening/lib/python3.10/site-packages/tensorflow_io-0.32.0-py3.10-macosx-13.3-arm64.egg/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/Alenka/.pyenv/versions/3.10.6/envs/GlobalGreening/lib/python3.10/site-packages/tensorflow_io-0.32.0-py3.10-macosx-13.3-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/Alenka/.pyenv/versions/3.10.6/envs/GlobalGreening/lib/python3.10/site-packages/tensorflow_io-0.32.0-py3.10-macosx-13.3-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/Alenka/.pyenv/versions/3.10.6/envs/GlobalGreening/lib/python3.10/site-packages/tensorflow_io-0.32.0-py3.10-macosx-13.3-arm64.egg/tensorflow_io/python/ops/__init__

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-06-13 14:18:01.975932: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-13 14:18:01.991685: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-13 14:18:01.993609: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-13 14:18:01.996762: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

## Key Params

Please make sure to create local folder structure in following way:

     |
     |---- raw_data
     |    |---- zoomed_photos
     |    |---- masks
     |
     |---- training_outputs
     |    |---- metrics
     |    |---- models
     |    |---- params

there will a folder for predictions as well, but will sort it out later

In [2]:
# parameters
DATA_SIZE=15415  
TRAIN_SIZE=8000     # how many images to train on
TRIAL_SIZE=TRAIN_SIZE + int(TRAIN_SIZE*0.2)

# model params
BATCH_SIZE = 1
PATIENCE = 4

# location
DATA_RUN = 'local_VM' #  select from these options:'gcp' local_VM' , 'local' 

# on VM workbench in GCP when loading data locally from VM  - change to what is has to be 
VM_dataset_folder = "/home/jupyter/GlobalGreening"  # need to change this 
VM_images_dir = "zoomed_photos"
VM_masks_dir = "masks"
VM_output_folder = "/home/jupyter/GlobalGreening/training_outputs" # need to change this 


In [3]:
# Define where to store model outputs

timestamp = time.strftime("%Y%m%d-%H%M%S")

if DATA_RUN == 'local_VM':
    model_path = os.path.join(VM_output_folder, "models", f"{timestamp}_InceptionResNetV2-UNet.h5")
    results_path = os.path.join(VM_output_folder, "metrics", f"{timestamp}_model_training.csv")
    model_metrics_plot_path = os.path.join(VM_output_folder, "metrics", f"{timestamp}_model_metrics_plot.png")
    predictions_dir = os.path.join(VM_output_folder, f"{timestamp}_predictions")
        
print("--------")
print(f' model path is {model_path}')
print("--------")
print(f' results path is {results_path}')
print("--------")
print(f' model metrics path is {model_metrics_plot_path}')
print("--------")
print(f'predictions folder is {predictions_dir}')

--------
 model path is /home/jupyter/GlobalGreening/training_outputs/models/20230613-162741_InceptionResNetV2-UNet.h5
--------
 results path is /home/jupyter/GlobalGreening/training_outputs/metrics/20230613-162741_model_training.csv
--------
 model metrics path is /home/jupyter/GlobalGreening/training_outputs/metrics/20230613-162741_model_metrics_plot.png
--------
predictions folder is /home/jupyter/GlobalGreening/training_outputs/20230613-162741_predictions


In [4]:
!pwd

/Users/Alenka/code/Alastair908/GlobalGreening/notebooks


In [6]:
# ***** NOT USED YET (folder to save predictions) ****** 

# # please make predictions directory based on what was printed out
# # check if any folders need to be created and adjust the below command 
# (see pwd path above)

# !mkdir -p ../training_outputs/20230610-132230_predictions

## Image masks

Value	Color	Description
- 10	#006400	Tree cover 
<p align="center"><div style="background-color: #006400; padding: 10px; "/></p>   
- 20	#ffbb22	Shrubland 
<p align="center"><div style="background-color: #ffbb22; padding: 10px; "/></p>   
- 30	#ffff4c	Grassland
<p align="center"><div style="background-color: #ffff4c; padding: 10px; "/></p>   
- 40	#f096ff	Cropland
<p align="center"><div style="background-color: #f096ff; padding: 10px; "/></p>  
- 50	#fa0000	Built-up
<p align="center"><div style="background-color: #fa0000; padding: 10px; "/></p>  
- 60	#b4b4b4	Bare / sparse vegetation
<p align="center"><div style="background-color: #b4b4b4; padding: 10px; "/></p>  
- 70	#f0f0f0	Snow and ice
<p align="center"><div style="background-color: #f0f0f0; padding: 10px; "/></p>  
- 80	#0064c8	Permanent water bodies
<p align="center"><div style="background-color: #0064c8; padding: 10px; "/></p>  
- 90	#0096a0	Herbaceous wetland
<p align="center"><div style="background-color: #0096a0; padding: 10px; "/></p>  
- 95	#00cf75	Mangroves
<p align="center"><div style="background-color: #00cf75; padding: 10px; "/></p>  
- 100	#fae6a0	Moss and lichen
<p align="center"><div style="background-color: #fae6a0; padding: 10px; "/></p> 

### Preparing labels

In [5]:
labels_dict = {"classes": [
{"title": "Tree cover", "r": 0 , "g": 0 , "b": 0 }, 
{"title": "Shrubland", "r": 0, "g": 0, "b": 0 }, 
{"title": "Grassland", "r": 0, "g": 0, "b": 0 }, 
{"title": "Cropland", "r": 0, "g": 0, "b": 0 }, 
{"title": "Built-up", "r": 0, "g": 0, "b": 0 }, 
{"title": "Bare, sparse vegetation", "r": 0, "g": 0, "b": 0 },
{"title": "Snow and ice", "r": 0, "g": 0, "b": 0 },
{"title": "Permanent water bodies", "r": 0, "g": 0, "b": 0 },
{"title": "Herbaceous wetland", "r": 0, "g": 0, "b": 0 },
{"title": "Mangroves", "r": 0, "g": 0, "b": 0 },
{"title": "Moss and lichen", "r": 0, "g": 0, "b": 0 }
]}


# loading correct rgb values from hex_color list based on ESA  
hex_colors_list = ['#006400', '#ffbb22', '#ffff4c', '#f096ff', '#fa0000',
                    '#b4b4b4', '#f0f0f0', '#0064c8', '#0096a0', '#00cf75', '#fae6a0']

# preparing label df with all information: classes, rgb values

def prepare_labels(labels_dict, hex_colors_list):

    labels_dict_df = pd.DataFrame(labels_dict['classes'])
    
    for i in range(len(hex_colors_list)):
        color = hex_colors_list[i].lstrip('#')
        r = int(color[0:2],16)
        g = int(color[2:4],16)
        b = int(color[4:6],16)
        labels_dict_df.at[i,'r'] = r
        labels_dict_df.at[i,'g'] = g
        labels_dict_df.at[i,'b'] = b
    
    return labels_dict_df

# preparing label codes (just rgb values in order of classes)

def prepare_label_codes(labels_dict, hex_colors_list):
    labels_dict_df = prepare_labels(labels_dict, hex_colors_list)
    
    label_codes = []
    r= np.asarray(labels_dict_df.r)
    g= np.asarray(labels_dict_df.g)
    b= np.asarray(labels_dict_df.b)

    for i in range(len(labels_dict_df)):
        label_codes.append(tuple([r[i], g[i], b[i]]))
    return label_codes

# preparing label names (just class names)

def prepare_label_names(labels_dict, hex_colors_list):
    labels_dict_df = prepare_labels(labels_dict, hex_colors_list)   
    label_names= list(labels_dict_df.title)
    return label_names

# preparing label codes (just rgb values in order of classes)

def prepare_label_codes(labels_dict, hex_colors_list):
    labels_dict_df = prepare_labels(labels_dict, hex_colors_list)
    
    label_codes = []
    r= np.asarray(labels_dict_df.r)
    g= np.asarray(labels_dict_df.g)
    b= np.asarray(labels_dict_df.b)

    for i in range(len(labels_dict_df)):
        label_codes.append(tuple([r[i], g[i], b[i]]))
    return label_codes

# preparing label names (just class names)

def prepare_label_names(labels_dict, hex_colors_list):
    labels_dict_df = prepare_labels(labels_dict, hex_colors_list)   
    label_names= list(labels_dict_df.title)
    return label_names

label_codes = prepare_label_codes(labels_dict, hex_colors_list)
label_names = prepare_label_names(labels_dict, hex_colors_list)

code2id = {v:k for k,v in enumerate(label_codes)}
id2code = {k:v for k,v in enumerate(label_codes)}

name2id = {v:k for k,v in enumerate(label_names)}
id2name = {k:v for k,v in enumerate(label_names)}

In [6]:
print(f'label codes are {label_codes}')
print('----')
print(f'label names are {label_names}')

label codes are [(0, 100, 0), (255, 187, 34), (255, 255, 76), (240, 150, 255), (250, 0, 0), (180, 180, 180), (240, 240, 240), (0, 100, 200), (0, 150, 160), (0, 207, 117), (250, 230, 160)]
----
label names are ['Tree cover', 'Shrubland', 'Grassland', 'Cropland', 'Built-up', 'Bare, sparse vegetation', 'Snow and ice', 'Permanent water bodies', 'Herbaceous wetland', 'Mangroves', 'Moss and lichen']


### Functions to One-hot Encode RGB Labels/Masks and Decoding Encoded Predictions

In [7]:
# one-hot encoder 
def rgb_to_onehot(rgb_mask_image, colormap = id2code):
    '''Function to one hot encode RGB mask labels
        Inputs: 
            rgb_image - image matrix (eg. 512 x 512 x 3 dimension numpy ndarray)
            colormap - dictionary of color to label id
        Output: One hot encoded image of dimensions (height x width x num_classes) where num_classes = len(colormap)
    '''
    num_classes = len(colormap)
    rgb_mask_to_encode = rgb_mask_image*255
#    print(f'shape of mask to encode is {rgb_mask_to_encode.shape}')
    # shape prepared for image size and channels = num of classes (instead of 3 RGB colors)
    shape = rgb_mask_to_encode.shape[:2]+(num_classes,)
#    print(f'prepared shape for encoded mask is {shape}')
    
    # encoded_image prepare array with right shaoe 
    encoded_mask = np.zeros( shape, dtype=np.int8 )
    for i, cls in enumerate(colormap):
        # image.reshape flattens and keeps 3 channels, then checks which pixels same as color in colormap
        # then change back to image size for each of 6 channels (based on colormap)
        encoded_mask[:,:,i] = np.all(rgb_mask_to_encode.reshape( (-1,3) ) == colormap[i], axis=1).reshape(shape[:2])
    
#    print(f'encoded mask shape is {encoded_mask.shape}')
    return encoded_mask

def onehot_to_rgb(onehot, colormap = id2code):
    '''Function to decode encoded mask labels
        Inputs: 
            onehot - one hot encoded image matrix (height x width x num_classes)
            colormap - dictionary of color to label id
        Output: Decoded RGB image (height x width x 3) 
    '''
    single_layer = np.argmax(onehot, axis=-1)
    output = np.zeros( onehot.shape[:2]+(3,) )
    for k in colormap.keys():
        output[single_layer==k] = colormap[k]
    return np.uint8(output)

#def preprocess_images (images):
#    images = images/255.
#    return images

#def encoding_masks(masks_dataset, colormap = id2code):
#    encoded_masks = []
##    print(f'range is {len(masks_dataset)}')
#    
#    for i in range(len(masks_dataset)): 
#        mask = masks_dataset[i]
#        encoded_mask = rgb_to_onehot(mask, colormap)
#        encoded_masks.append(encoded_mask)
#    
#    encoded_masks = np.array(encoded_masks) 
#    return encoded_masks

## Loading the Data

### Split train, val, test

In [8]:
with open("images_path_train.txt", "r") as f:
    images_path_train = json.load(f)
    
with open("images_path_val.txt", "r") as f:
    images_path_val = json.load(f)

with open("images_path_test.txt", "r") as f:
    images_path_test = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'images_path_train.txt'

In [11]:
print(len(images_path_train), images_path_train[0:20])
print('---')
print(len(images_path_val), images_path_val[0:20])
print('---')
print(len(images_path_test), images_path_test[0:20])

8000 ['zoomed_photos/image10952_-104.05_39.05.png', 'zoomed_photos/image2133_-108.06_39.78.png', 'zoomed_photos/image14647_-102.38_40.29.png', 'zoomed_photos/image11380_-103.83_37.26.png', 'zoomed_photos/image9141_-104.86_37.98.png', 'zoomed_photos/image4649_-106.91_38.84.png', 'zoomed_photos/image5620_-106.48_40.16.png', 'zoomed_photos/image5142_-106.69_39.82.png', 'zoomed_photos/image5266_-106.61_37.09.png', 'zoomed_photos/image1551_-108.32_39.01.png', 'zoomed_photos/image1371_-108.4_39.35.png', 'zoomed_photos/image12154_-103.49_38.2.png', 'zoomed_photos/image6759_-105.97_40.63.png', 'zoomed_photos/image9417_-104.73_37.73.png', 'zoomed_photos/image7712_-105.5_37.17.png', 'zoomed_photos/image3342_-107.51_39.22.png', 'zoomed_photos/image14668_-102.34_37.17.png', 'zoomed_photos/image12790_-103.19_37.26.png', 'zoomed_photos/image8785_-105.03_38.84.png', 'zoomed_photos/image8339_-105.24_39.86.png']
---
1600 ['zoomed_photos/image3070_-107.63_39.65.png', 'zoomed_photos/image14869_-102.25_37

In [12]:
paths_train = [(image_path, image_path.replace('image','mask').replace('.png','.npy').replace("zoomed_photos","masks")) for image_path in images_path_train]
paths_val = [(image_path, image_path.replace('image','mask').replace('.png','.npy').replace("zoomed_photos","masks")) for image_path in images_path_val]
paths_test = [(image_path, image_path.replace('image','mask').replace('.png','.npy').replace("zoomed_photos","masks")) for image_path in images_path_test]

In [13]:
print(len(paths_train), paths_train[0:3])
print('---')
print(len(paths_val), paths_val[0:3])
print('---')
print(len(paths_test), paths_test[0:3])

8000 [('zoomed_photos/image10952_-104.05_39.05.png', 'masks/mask10952_-104.05_39.05.npy'), ('zoomed_photos/image2133_-108.06_39.78.png', 'masks/mask2133_-108.06_39.78.npy'), ('zoomed_photos/image14647_-102.38_40.29.png', 'masks/mask14647_-102.38_40.29.npy')]
---
1600 [('zoomed_photos/image3070_-107.63_39.65.png', 'masks/mask3070_-107.63_39.65.npy'), ('zoomed_photos/image14869_-102.25_37.73.png', 'masks/mask14869_-102.25_37.73.npy'), ('zoomed_photos/image9990_-104.47_38.11.png', 'masks/mask9990_-104.47_38.11.npy')]
---
5815 [('zoomed_photos/image8174_-105.33_40.84.png', 'masks/mask8174_-105.33_40.84.npy'), ('zoomed_photos/image1009_-108.57_39.95.png', 'masks/mask1009_-108.57_39.95.npy'), ('zoomed_photos/image4815_-106.82_37.9.png', 'masks/mask4815_-106.82_37.9.npy')]


### Functions to load train, val, test into tf.dataset

In [9]:
def load_mask(mask_path):
    data = np.load(mask_path.numpy().decode("utf-8"))
    data = rgb_to_onehot(data, id2code)
    data = tf.convert_to_tensor(data, dtype=tf.float32)
    return data

def read_data_from_images(paths):
#    image_path = str(image_path).replace("tf.Tensor(b'","").replace("', shape=(), dtype=string)","")
#    mask_path = image_path.replace('image','mask').replace('.png','.npy').replace("zoomed_photos","masks")
#    image = tf.io.image.read_image()  # this part from Ollie

    raw = tf.io.read_file(paths[0])
    image = tf.io.decode_png(raw, channels=3)
    image = tf.cast(image, dtype=tf.float32)
    image = image / 255.0
 #   image = tf.reshape(image, (512, 512, 3))
     
    mask = tf.py_function(load_mask, inp=[paths[1]], Tout=tf.float32)
    mask = tf.reshape(mask, (512, 512, 11))
    return (image, mask)

In [15]:
ds_train = tf.data.Dataset.from_tensor_slices(paths_train)

In [16]:
ds_train = ds_train.map(lambda x: read_data_from_images(x))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [17]:
ds_train

<MapDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(512, 512, 11), dtype=tf.float32, name=None))>

In [18]:
ds_val = tf.data.Dataset.from_tensor_slices(paths_val)

In [19]:
ds_val = ds_val.map(lambda x: read_data_from_images(x))

In [20]:
ds_val

<MapDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(512, 512, 11), dtype=tf.float32, name=None))>

In [21]:
ds_test = tf.data.Dataset.from_tensor_slices(paths_test)

In [22]:
ds_test = ds_test.map(lambda x: read_data_from_images(x))

In [23]:
ds_test

<MapDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(512, 512, 11), dtype=tf.float32, name=None))>

## Model

### Setting up InceptionResNetV2 UNet

In [10]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_inception_resnetv2_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained InceptionResNetV2 Model """
    encoder = InceptionResNetV2(include_top=False, weights="imagenet", input_tensor=inputs)

    """ Encoder """
    s1 = encoder.get_layer("input_1").output           ## (512 x 512)

    s2 = encoder.get_layer("activation").output        ## (255 x 255)
    s2 = ZeroPadding2D(( (1, 0), (1, 0) ))(s2)         ## (256 x 256)

    s3 = encoder.get_layer("activation_3").output      ## (126 x 126)
    s3 = ZeroPadding2D((1, 1))(s3)                     ## (128 x 128)

    s4 = encoder.get_layer("activation_74").output      ## (61 x 61)
    s4 = ZeroPadding2D(( (2, 1),(2, 1) ))(s4)           ## (64 x 64)

    """ Bridge """
    b1 = encoder.get_layer("activation_161").output     ## (30 x 30)
    b1 = ZeroPadding2D((1, 1))(b1)                      ## (32 x 32)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                     ## (64 x 64)
    d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
    d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
    d4 = decoder_block(d3, s1, 64)                      ## (512 x 512)
    
    """ Output """
    dropout = Dropout(0.3)(d4)
    outputs = Conv2D(11, 1, padding="same", activation="softmax")(dropout)

    model = Model(inputs, outputs, name="InceptionResNetV2-UNet")
    return model

In [11]:
K.clear_session()

def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

model = build_inception_resnetv2_unet(input_shape = (512, 512, 3))
model.compile(optimizer=Adam(lr = 0.0001), loss='categorical_crossentropy', metrics=[dice_coef, "accuracy"])
model.summary()

Model: "InceptionResNetV2-UNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 255, 255, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 255, 255, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                            

                                                                                                  
 conv2d_10 (Conv2D)             (None, 61, 61, 96)   82944       ['activation_9[0][0]']           
                                                                                                  
 conv2d_11 (Conv2D)             (None, 61, 61, 64)   12288       ['average_pooling2d[0][0]']      
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 61, 61, 96)  288         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 61, 61, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
          

 conv2d_21 (Conv2D)             (None, 61, 61, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 batch_normalization_21 (BatchN  (None, 61, 61, 32)  96          ['conv2d_21[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_21 (Activation)     (None, 61, 61, 32)   0           ['batch_normalization_21[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 61, 61, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_22 (Conv2D)             (None, 61, 61, 48)   13824       ['activation_21[0][0]']          
          

 batch_normalization_26 (BatchN  (None, 61, 61, 32)  96          ['conv2d_26[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_29 (BatchN  (None, 61, 61, 64)  192         ['conv2d_29[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_24 (Activation)     (None, 61, 61, 32)   0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 61, 61, 32)   0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activatio

 conv2d_37 (Conv2D)             (None, 61, 61, 32)   10240       ['block35_4_ac[0][0]']           
                                                                                                  
 conv2d_40 (Conv2D)             (None, 61, 61, 48)   13824       ['activation_39[0][0]']          
                                                                                                  
 batch_normalization_37 (BatchN  (None, 61, 61, 32)  96          ['conv2d_37[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_40 (BatchN  (None, 61, 61, 48)  144         ['conv2d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

...

                                                                                                  
 batch_normalization_134 (Batch  (None, 30, 30, 160)  480        ['conv2d_134[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_134 (Activation)    (None, 30, 30, 160)  0           ['batch_normalization_134[0][0]']
                                                                                                  
 conv2d_132 (Conv2D)            (None, 30, 30, 192)  208896      ['block17_14_ac[0][0]']          
                                                                                                  
 conv2d_135 (Conv2D)            (None, 30, 30, 192)  215040      ['activation_134[0][0]']         
                                                                                                  
 batch_nor

                                                                                                  
 activation_142 (Activation)    (None, 30, 30, 160)  0           ['batch_normalization_142[0][0]']
                                                                                                  
 conv2d_140 (Conv2D)            (None, 30, 30, 192)  208896      ['block17_16_ac[0][0]']          
                                                                                                  
 conv2d_143 (Conv2D)            (None, 30, 30, 192)  215040      ['activation_142[0][0]']         
                                                                                                  
 batch_normalization_140 (Batch  (None, 30, 30, 192)  576        ['conv2d_140[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_nor

 conv2d_148 (Conv2D)            (None, 30, 30, 192)  208896      ['block17_18_ac[0][0]']          
                                                                                                  
 conv2d_151 (Conv2D)            (None, 30, 30, 192)  215040      ['activation_150[0][0]']         
                                                                                                  
 batch_normalization_148 (Batch  (None, 30, 30, 192)  576        ['conv2d_148[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_151 (Batch  (None, 30, 30, 192)  576        ['conv2d_151[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activatio

 conv2d_transpose (Conv2DTransp  (None, 64, 64, 512)  590336     ['zero_padding2d_3[0][0]']       
 ose)                                                                                             
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 64, 64, 256)  0          ['activation_74[0][0]']          
 D)                                                                                               
                                                                                                  
 concatenate (Concatenate)      (None, 64, 64, 768)  0           ['conv2d_transpose[0][0]',       
                                                                  'zero_padding2d_2[0][0]']       
                                                                                                  
 conv2d_203 (Conv2D)            (None, 64, 64, 512)  3539456     ['concatenate[0][0]']            
          

 conv2d_209 (Conv2D)            (None, 512, 512, 64  38656       ['concatenate_3[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_209 (Batch  (None, 512, 512, 64  256        ['conv2d_209[0][0]']             
 Normalization)                 )                                                                 
                                                                                                  
 activation_209 (Activation)    (None, 512, 512, 64  0           ['batch_normalization_209[0][0]']
                                )                                                                 
                                                                                                  
 conv2d_210 (Conv2D)            (None, 512, 512, 64  36928       ['activation_209[0][0]']         
          

### Modelling

In [12]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.0001, 60)

lr_scheduler = LearningRateScheduler(
    exponential_decay_fn,
    verbose=1
)

checkpoint = ModelCheckpoint(
    filepath = model_path,
    save_best_only = True, 
#     save_weights_only = False,
    monitor = 'val_loss', 
    mode = 'auto', 
    verbose = 1
)

earlystop = EarlyStopping(
    monitor = 'val_loss', 
    min_delta = 0.001, 
    patience = PATIENCE, 
    mode = 'auto', 
    verbose = 1,
    restore_best_weights = True
)

csvlogger = CSVLogger(
    filename = results_path,
    separator = ",",
    append = False
)

callbacks = [checkpoint, earlystop, csvlogger, lr_scheduler]

In [ ]:
### create batch (not cache yet)
BATCH_SIZE = 1
ds = ds.batch(1)
# ds = ds.cache(".

In [ ]:
history = model.fit(ds)

In [ ]:
model.save(model_path)

In [ ]:
#history = model.fit(
#    ds,
#    batch_size=BATCH_SIZE,
#    validation_split = 0.2, 
#    epochs = 50,
#    callbacks=callbacks, 
#    verbose=1
#)

### Loading results and making predictions

In [ ]:
# if model doesn't finish, this will not work
# then we load it from the cvs result

df_result = pd.DataFrame(history.history)
df_result

In [ ]:
model.load_weights(model_path)

In [ ]:
pred_all= model.predict(X_test)

In [ ]:
# visualized all prediction and save the combo: 
#    - image, ground truth mask, predicted mask
# as a file

np.shape(pred_all)

count = 0
 
for j in range(0,np.shape(pred_all)[0]):
    count += 1
    fig = plt.figure(figsize=(20,8))

    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(batch_img[j])
    ax1.set_title('Input Image', fontdict={'fontsize': 16, 'fontweight': 'medium'})
    ax1.grid(False)

    ax2 = fig.add_subplot(1,3,2)
    ax2.set_title('Ground Truth Mask', fontdict={'fontsize': 16, 'fontweight': 'medium'})
    ax2.imshow(onehot_to_rgb(batch_mask[j],id2code))
    ax2.grid(False)

    ax3 = fig.add_subplot(1,3,3)
    ax3.set_title('Predicted Mask', fontdict={'fontsize': 16, 'fontweight': 'medium'})
    ax3.imshow(onehot_to_rgb(pred_all[j],id2code))
    ax3.grid(False)

#    plt.savefig('./predictions/prediction_{}.png'.format(count), facecolor= 'w', transparent= False, bbox_inches= 'tight', dpi= 200)
    plt.show()